<a href="https://colab.research.google.com/github/alangttl2/Proyecto_1-Algoritmo_PageRank/blob/main/Proyecto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
######################### Alan Ledezma Puente #################################
# Este código lo que hace es imitar el algoritmo PageRank (que se usa en Google)
# para poder clasificar las páginas por importancia, por medio de sus citados
# que tienen de otras páginas, por medio de matrices y calculos que permiten
# saber el peso de cada una y así poder sacar su importancia, en este código
# es lo que se hará viendo la importancia de páginas cambiando vectores para
# poder determinar la importancia de cada una de las páginas.

import numpy as np
import pandas as pd

class Grafo:
    def __init__(self, nodos, arcos): # Construimos nodos y arcos para la matriz
        """Clase para construir la matriz de adyacencia binaria y contar citas"""
        self.nodos = nodos
        self.matriz = np.zeros((len(nodos), len(nodos)))

        # Llenamos la matriz de adyacencia con los arcos
        for origen, destino in arcos:
            self.matriz[origen - 1, destino - 1] = 1  # Ajustamos a base 0

        print("Matriz de adyacencia binaria:")
        print(self.matriz) # Print para poder ver la mtriz binaria

class Redes:
    def __init__(self, grafo):
        """Clase para construir la matriz de pesos usando el grafo"""
        self.matriz_citados = np.zeros_like(grafo.matriz, dtype=float)

        # Calculamos la matriz de pesos como la proporción en cada fila
        for i in range(len(grafo.matriz)):
            if grafo.matriz[i, :].sum() > 0:# Verifica que la fila no esté vacía
                self.matriz_citados[i, :] = grafo.matriz[i, :] / \
                                            grafo.matriz[i, :].sum()

        print("Matriz de pesos:")
        print(self.matriz_citados)

class PageRank:
    def __init__(self, matriz_citados, tol=1e-6, max_iter=100):
        """Calcula el PageRank usando una matriz de citados y un factor de
          amortiguación d"""
        self.matriz_citados = matriz_citados
        self.tol = tol # Asegura que el algoritmo pare cuando ha alcanzado una
                       # solución suficientemente precisa
        self.max_iter = max_iter # Limita el número máximo de iteraciones para
                                 # evitar bucles infinitos si no converge
        self.n = matriz_citados.shape[0] # Para tener la matriz en forma 0
        self.U = np.ones((self.n, self.n)) / self.n  # Matriz U de probabilidad
                                                     # uniforme

    def calcular_pagerank(self, d):
        """Vector inicial uniforme"""
        pi = np.ones(self.n) / self.n
        for _ in range(self.max_iter):
            # Multiplicación iterativa
            pi_nuevo = pi @ (d * self.matriz_citados + (1 - d) * self.U)
            # Verificamos la convergencia
            if np.linalg.norm(pi_nuevo - pi) < self.tol:
                break # Hace las operaciones lineales maximo 100 veces
            pi = pi_nuevo
        return pi # Regresa pi con los calculos ya hechos

# Función para construir el grafo a partir de un archivo de Excel y contestar
# la primera pregunta.
def construccion_matriz(archivo_datos):
    df = pd.read_excel(archivo_datos)
    nodos = list(df["Index"].astype(int)) # modificamos la comlumna a enteros
    arcos = []
    terminacion_ru = [] # Lista donde estaran los elementos .ru almacenados

    for i in range(len(df)):# Para poder identificar cada uno de terminación .ru
        citados = str(df["Cited by"][i])
        url = str(df["Website"][i])
        terminacion_ru.append(url.endswith('.ru'))

        if citados and citados != "nan": # Verifica si hay nodos citados
            for j in citados.split(','):
                arcos.append([int(df["Index"][i]), int(j.strip())])

    grafo = Grafo(nodos, arcos)
    return grafo, terminacion_ru # Regresa la matriz adyacente de los valores .ru

# Para responder la segunda pregunta
def calcular_pi_ru(terminacion_ru):
    """Calcula el vector pi para páginas con terminación .ru"""
    s = sum(terminacion_ru)
    return np.array([1/s if es_ru else 0 for es_ru in terminacion_ru])

def main(archivo_datos):
    # Construimos el grafo para obtener la matriz y las páginas con terminación
    # .ru
    grafo, terminacion_ru = construccion_matriz(archivo_datos)

    # Creamos la matriz de pesos usando el grafo
    matriz_citados = Redes(grafo)

    # Usar la clase PageRank con la matriz de citados
    pagerank = PageRank(matriz_citados.matriz_citados)
    # d=0.85 para la primera pregunta(usamos 0.85 porque es el valor de
    # amortiguación porque es el generalmente se supone):
    vector_importancia = pagerank.calcular_pagerank(d=0.85)
    print("Vector de importancia (PageRank):", vector_importancia)

    # Identificar la página más importante en el PageRank inicial
    pagina_mas_importante = np.argmax(vector_importancia)
    # Ajuste para índice base 1
    print("La página más importante es:", pagina_mas_importante + 1)

    # Calcular el vector pi para páginas con terminación .ru
    pi_ru = calcular_pi_ru(terminacion_ru)
    print("Vector pi para .ru:", pi_ru)

    # Comparación de importancia
    es_mas_importante_ru = vector_importancia[pagina_mas_importante] == \
                                                max(vector_importancia * pi_ru)
    print("La pagina más importante es la 9:", es_mas_importante_ru)

    # Tercera pregunta: probar diferentes valores de d
    valores_d = [0.5, 0.85, 1]
    for d in valores_d:
        vector_importancia = pagerank.calcular_pagerank(d)
        print(f"Vector de importancia con d={d}:", vector_importancia)
        pagina_mas_importante = np.argmax(vector_importancia)
        # Ajuste a índice base 1
        print(f"La página más importante con d={d} es:", \
              pagina_mas_importante + 1)

# Llamada a la función main con la ruta del archivo de datos
archivo_datos = "/content/drive/MyDrive/Web (3).xlsx"
# Implementación del main para archivo_datos
if __name__ == "__main__":
    main(archivo_datos)

Matriz de adyacencia binaria:
[[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0